<a href="https://colab.research.google.com/github/dorsaalh/Cryptocurrency_Price_Prediction/blob/main/Finbert_768.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pathlib import Path
import shutil
import os
import logging
from textblob import TextBlob
import transformers
from transformers import AutoModelForSequenceClassification
import sys
sys.path.append('/content/drive/MyDrive/finbert/finBERT-master')
from finbert.finbert import *
#import finbert.utils as tools
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data_for_finbert.csv")

In [ ]:
data

,Unnamed: 0,text,account,label
0,0,"RT @ArmstrongDrew: Well, Pfizer's having a hell of a week. First earnings, now this (stock up 11% premarket) $PFE\n\nhttps://t.co/9lMZd3m3Lv",Bloomberg,0
1,1,RT @markets: U.S. stock futures rise https://t.co/MsndJFmn2i https://t.co/lHskGoYdSs,Bloomberg,0
2,2,"The U.S. labor market got back on track last month with a larger-than-forecast payrolls gain, indicating greater pr… https://t.co/Lyx3YzCuNh",Bloomberg,0
3,3,Follow the latest on the October jobs report here https://t.co/g3G65YXIQP https://t.co/0VnXlrQCA5,Bloomberg,0
4,4,"It will take years to fully understand the pandemic’s ripple effects on our health, economy and society. But early… https://t.co/9KJgp30Cte",Bloomberg,0
...,...,...,...,...
2547,2547,009🔺 @BITCOIN_AL_DIA https://t.co/UtwYD9CnlD,Binance,0
2548,2548,008🔺 @REALDIEGOVALDEZ https://t.co/h25WTLtegd,Binance,0
2549,2549,007🔺 @kursadbucak https://t.co/PVIlyhrbMj,Binance,0
2550,2550,006🔺 @healthy_pockets https://t.co/fitDIMC8ke,Binance,0


In [ ]:
text = data.text.values

In [ ]:
len(text)

2552

In [ ]:
cl_path = 'ProsusAI/finbert'
finbert = AutoModelForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
 class Model_768(transformers.models.bert.modeling_bert.BertForSequenceClassification):

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]

        
        return pooled_output

In [ ]:
model = Model_768.from_pretrained(cl_path, cache_dir=None, num_labels=3)

In [ ]:
model.bert.encoder.layer[10].intermediate.dense.weight

Parameter containing:
tensor([[-0.0718, -0.0310,  0.0214,  ..., -0.0296,  0.0285,  0.0754],
        [ 0.0736,  0.0472, -0.0160,  ..., -0.0956,  0.0134, -0.0311],
        [ 0.0158,  0.0003,  0.0028,  ...,  0.0029, -0.0329, -0.0027],
        ...,
        [ 0.0586, -0.0044,  0.0707,  ..., -0.0213, -0.0047, -0.0098],
        [-0.0325, -0.0271, -0.0084,  ...,  0.0234, -0.0085,  0.0163],
        [-0.0166, -0.0063,  0.0339,  ..., -0.0126,  0.0050,  0.0667]],
       requires_grad=True)

In [ ]:
model.classifier

Linear(in_features=768, out_features=3, bias=True)

In [ ]:
text = """
Bitcoin price has increased unexpectedly.  
Bitcoin price has decreased.
"""

In [ ]:
def my_predict(text, model, write_to_csv=False, path=None, use_gpu=False, gpu_name='cuda:0', batch_size=5):
    """
    Predict sentiments of sentences in a given text. The function first tokenizes sentences, make predictions and write
    results.
    Parameters
    ----------
    text: string
        text to be analyzed
    model: BertForSequenceClassification
        path to the classifier model
    write_to_csv (optional): bool
    path (optional): string
        path to write the string
    use_gpu: (optional): bool 
        enables inference on GPU
    gpu_name: (optional): string
        multi-gpu support: allows specifying which gpu to use
    batch_size: (optional): int
        size of batching chunks
    """
    model.eval()

    sentences = sent_tokenize(text)
    print(sentences)
    print(len(sentences))

    device = gpu_name if use_gpu and torch.cuda.is_available() else "cpu"
    logging.info("Using device: %s " % device)
    label_list = ['positive', 'negative', 'neutral']
    label_dict = {0: 'positive', 1: 'negative', 2: 'neutral'}
    # result = pd.DataFrame(columns=['sentence', 'logit'])
    logit_list = []
    for batch in chunks(sentences, batch_size):
        examples = [InputExample(str(i), sentence) for i, sentence in enumerate(batch)]
        features = convert_examples_to_features(examples, label_list, 64, tokenizer)
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long).to(device)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long).to(device)
        all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long).to(device)
        with torch.no_grad():
            model = model.to(device)

            # logging.info(logits)
            
            # logits = softmax(np.array(logits.cpu()))
            # sentiment_score = pd.Series(logits[:, 0] - logits[:, 1])
            # predictions = np.squeeze(np.argmax(logits, axis=1))

            # batch_result = {'sentence': batch[0]}
            logit_list.append(np.array(model(all_input_ids, all_attention_mask, all_token_type_ids).cpu()))
                            # 'prediction': predictions,
                            # 'sentiment_score': sentiment_score}

            # batch_result = pd.DataFrame(batch_result,columns=['sentence'])
            # result = pd.concat([result, batch_result], ignore_index=True)

    # result['prediction'] = result.prediction.apply(lambda x: label_dict[x])
    # if write_to_csv:
        # result.to_csv(path, sep=',', index=False)

    return np.vstack(logit_list)